In [1]:
import csv
import psycopg2
import os

In [2]:
connection = psycopg2.connect(dbname="tracking", user="postgres", password="Heartmole-1337")

In [3]:
missing_locations = set()
def get_location_id(original_location):
    location = original_location.split("\n")[-1]
    cursor = connection.cursor()
    the_id = 0
    try:
        cursor.execute("SELECT id FROM tracking_location WHERE name = %s", (location,))
        result = cursor.fetchone()
        if result is None:
            missing_locations.add(location)
        else:
            the_id = result[0]
    finally:
        cursor.close()
        return the_id

In [4]:
def insert_container(data):
    cursor = connection.cursor()
    try:
        result = cursor.execute("INSERT INTO tracking_container (code, carrier_id, origin_id, destination_id, "
                                "processed) VALUES (%s, %s, %s, %s, %s)", data)
    finally:
        cursor.close()

In [5]:
def insert_containers(database, carrier):
    filename = "containers/{}-{}.csv".format(database, carrier)
    if os.path.exists(filename):
        with open(filename, "r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                first_location = get_location_id(row["first_location"])
                last_location  = get_location_id(row["last_location"])
                if (first_location > 0) and (last_location > 0):
                    data = (row["container"], int(row["carrier"]), first_location, last_location, True)
                    insert_container(data)
    else:
        print("File {} does not exist".format(filename))
    # If everything goes ok, commit
    connection.commit()

In [6]:
insert_containers("scraper2", "Hapag-Lloyd")
insert_containers("scraper2", "Maersk")
insert_containers("scraper2", "Evergreen")

Location JIANGMEN not found...
Location PITHAMPUR not found...
Location TAOYUAN not found...
Location SURABAYA not found...
Location JIANGMEN not found...
Location WUHU not found...
Location TOLUCA, MEX not found...
Location TAOYUAN not found...
Location CIVITAVECCHIA not found...
Location ZHUHAI not found...
Location CIVITAVECCHIA not found...
Location ZHUHAI not found...
Location BANG PHLI not found...
Location CORINTO not found...
Location CORINTO not found...
Location BELAWAN not found...
Location CORINTO not found...
Location CORINTO not found...
Location BELAWAN not found...
Location BELAWAN not found...
Location BELAWAN not found...
Location ANKLESHWAR not found...
Location CINCINNATI, OH not found...
Location CINCINNATI, OH not found...
Location GRAND RAPIDS, MI not found...
Location LOUISVILLE, KY not found...
Location LOUISVILLE, KY not found...
Location NEWARK, NJ not found...
Location Inchon, Korea, South not found...
Location Amman, Jordan not found...
Location Jawaharlal 

In [7]:
connection.close()